In [2]:
import sqlite3
import pandas as pd

# Connect to the uploaded SQLite database
conn = sqlite3.connect('database.sqlite')

In [4]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,Salaries


In [6]:
# Top 10 highest paid employees in 2011 (based on TotalPay)
query1 = """
SELECT EmployeeName, JobTitle, TotalPay
FROM Salaries
WHERE Year = 2011
ORDER BY TotalPay DESC
LIMIT 10;
"""
pd.read_sql(query1, conn)


,EmployeeName,JobTitle,TotalPay
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),335279.91
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,332343.61
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",326373.19
5,DAVID SULLIVAN,ASSISTANT DEPUTY CHIEF II,316285.74
6,ALSON LEE,"BATTALION CHIEF, (FIRE DEPARTMENT)",315981.05
7,DAVID KUSHNER,DEPUTY DIRECTOR OF INVESTMENTS,307899.46
8,MICHAEL MORRIS,"BATTALION CHIEF, (FIRE DEPARTMENT)",303427.55
9,JOANNE HAYES-WHITE,"CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)",302377.73


In [8]:
# Create a temp view for job average pay
query2 = """
WITH JobAvg AS (
    SELECT JobTitle, AVG(TotalPay) AS AvgJobPay
    FROM Salaries
    GROUP BY JobTitle
)
SELECT s.EmployeeName, s.JobTitle, s.TotalPay, j.AvgJobPay
FROM Salaries s
INNER JOIN JobAvg j ON s.JobTitle = j.JobTitle
WHERE s.Year = 2011
ORDER BY s.TotalPay DESC
LIMIT 10;
"""
pd.read_sql(query2, conn)


,EmployeeName,JobTitle,TotalPay,AvgJobPay
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,399211.275000
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,211783.893243
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),335279.91,211783.893243
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,332343.61,145073.492500
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",326373.19,279359.560000
5,DAVID SULLIVAN,ASSISTANT DEPUTY CHIEF II,316285.74,228724.506000
6,ALSON LEE,"BATTALION CHIEF, (FIRE DEPARTMENT)",315981.05,216655.536889
7,DAVID KUSHNER,DEPUTY DIRECTOR OF INVESTMENTS,307899.46,307899.460000
8,MICHAEL MORRIS,"BATTALION CHIEF, (FIRE DEPARTMENT)",303427.55,216655.536889
9,JOANNE HAYES-WHITE,"CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)",302377.73,302377.730000


In [10]:
# Employees earning more than the average total pay in 2011
query3 = """
SELECT EmployeeName, TotalPay
FROM Salaries
WHERE Year = 2011
AND TotalPay > (
    SELECT AVG(TotalPay) FROM Salaries WHERE Year = 2011
)
ORDER BY TotalPay DESC
LIMIT 10;
"""
pd.read_sql(query3, conn)


,EmployeeName,TotalPay
0,NATHANIEL FORD,567595.43
1,GARY JIMENEZ,538909.28
2,ALBERT PARDINI,335279.91
3,CHRISTOPHER CHONG,332343.61
4,PATRICK GARDNER,326373.19
5,DAVID SULLIVAN,316285.74
6,ALSON LEE,315981.05
7,DAVID KUSHNER,307899.46
8,MICHAEL MORRIS,303427.55
9,JOANNE HAYES-WHITE,302377.73


In [12]:
# Average and total pay per year
query4 = """
SELECT Year, 
       SUM(TotalPay) AS TotalPaySum,
       AVG(TotalPay) AS AveragePay
FROM Salaries
GROUP BY Year
ORDER BY Year;
"""
pd.read_sql(query4, conn)


,Year,TotalPaySum,AveragePay
0,2011,2.594195e+09,71744.103871
1,2012,2.724848e+09,74113.262265
2,2013,2.918656e+09,77611.443142
3,2014,2.876911e+09,75463.918140


In [18]:
import shutil

# Copy the database to a writable location
shutil.copy('database.sqlite', 'writable_database.sqlite')

# Reconnect to the new writable database
conn = sqlite3.connect('writable_database.sqlite')

In [20]:
# Now this should work
conn.execute("""
CREATE VIEW IF NOT EXISTS HighEarners AS
SELECT EmployeeName, JobTitle, TotalPay, Year
FROM Salaries
WHERE TotalPay > 300000;
""")

# View the results
pd.read_sql("SELECT * FROM HighEarners LIMIT 5;", conn)


,EmployeeName,JobTitle,TotalPay,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",326373.19,2011


In [22]:
# Create indexes on frequently used columns
conn.execute("CREATE INDEX IF NOT EXISTS idx_year ON Salaries(Year);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_totalpay ON Salaries(TotalPay);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_jobtitle ON Salaries(JobTitle);")

In [24]:
# We'll perform deeper data analysis using the Salaries table.

# 1. Most common job titles
query_job_titles = """
SELECT JobTitle, COUNT(*) as Count
FROM Salaries
GROUP BY JobTitle
ORDER BY Count DESC
LIMIT 10;
"""
most_common_jobs = pd.read_sql(query_job_titles, conn)

# 2. Yearly total payments trend
query_yearly_trend = """
SELECT Year, SUM(TotalPay) as TotalPayment
FROM Salaries
GROUP BY Year
ORDER BY Year;
"""
yearly_payments = pd.read_sql(query_yearly_trend, conn)

# 3. Top 10 earners by TotalPayBenefits
query_top_earners = """
SELECT EmployeeName, JobTitle, TotalPayBenefits
FROM Salaries
ORDER BY TotalPayBenefits DESC
LIMIT 10;
"""
top_earners = pd.read_sql(query_top_earners, conn)

# 4. Average BasePay, OvertimePay, OtherPay by Year
query_avg_pay_types = """
SELECT Year,
       AVG(BasePay) as AvgBasePay,
       AVG(OvertimePay) as AvgOvertimePay,
       AVG(OtherPay) as AvgOtherPay
FROM Salaries
GROUP BY Year
ORDER BY Year;
"""
avg_pay_types = pd.read_sql(query_avg_pay_types, conn)

# 5. Departments with highest total pay (based on JobTitle keywords)
query_dept_total = """
SELECT JobTitle, SUM(TotalPay) as TotalPay
FROM Salaries
WHERE JobTitle LIKE '%FIRE%' OR JobTitle LIKE '%POLICE%'
GROUP BY JobTitle
ORDER BY TotalPay DESC
LIMIT 10;
"""
dept_total = pd.read_sql(query_dept_total, conn)

import ace_tools as tools; tools.display_dataframe_to_user(name="Most Common Job Titles", dataframe=most_common_jobs)
tools.display_dataframe_to_user(name="Yearly Total Payments", dataframe=yearly_payments)
tools.display_dataframe_to_user(name="Top Earners", dataframe=top_earners)
tools.display_dataframe_to_user(name="Average Pay Types by Year", dataframe=avg_pay_types)
tools.display_dataframe_to_user(name="Top Paying Departments", dataframe=dept_total)


ModuleNotFoundError: No module named 'ace_tools'

In [28]:
query_job_titles = """
SELECT JobTitle, COUNT(*) as Count
FROM Salaries
GROUP BY JobTitle
ORDER BY Count DESC
LIMIT 10;
"""
most_common_jobs = pd.read_sql(query_job_titles, conn)
most_common_jobs

,JobTitle,Count
0,Transit Operator,7036
1,Special Nurse,4389
2,Registered Nurse,3736
3,Public Svc Aide-Public Works,2518
4,Police Officer 3,2421
5,Custodian,2418
6,TRANSIT OPERATOR,2388
7,Firefighter,2359
8,Recreation Leader,1971
9,Patient Care Assistant,1945


In [30]:
query_yearly_trend = """
SELECT Year, SUM(TotalPay) as TotalPayment
FROM Salaries
GROUP BY Year
ORDER BY Year;
"""
yearly_payments = pd.read_sql(query_yearly_trend, conn)
yearly_payments

,Year,TotalPayment
0,2011,2.594195e+09
1,2012,2.724848e+09
2,2013,2.918656e+09
3,2014,2.876911e+09


In [32]:
query_top_earners = """
SELECT EmployeeName, JobTitle, TotalPayBenefits
FROM Salaries
ORDER BY TotalPayBenefits DESC
LIMIT 10;
"""
top_earners = pd.read_sql(query_top_earners, conn)
top_earners

,EmployeeName,JobTitle,TotalPayBenefits
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28
2,David Shinn,Deputy Chief 3,510732.68
3,Amy P Hart,Asst Med Examiner,479652.21
4,William J Coaker Jr.,Chief Investment Officer,436224.36
5,Gregory P Suhr,Chief of Police,425815.28
6,Joanne M Hayes-White,"Chief, Fire Department",422353.40
7,Gregory P Suhr,Chief of Police,418019.22
8,Joanne M Hayes-White,"Chief, Fire Department",417435.10
9,Ellen G Moffatt,Asst Med Examiner,415767.94


In [34]:
query_avg_pay_types = """
SELECT Year,
       AVG(BasePay) as AvgBasePay,
       AVG(OvertimePay) as AvgOvertimePay,
       AVG(OtherPay) as AvgOtherPay
FROM Salaries
GROUP BY Year
ORDER BY Year;
"""
avg_pay_types = pd.read_sql(query_avg_pay_types, conn)
avg_pay_types

,Year,AvgBasePay,AvgOvertimePay,AvgOtherPay
0,2011,63595.956517,4531.065429,3617.081926
1,2012,65436.406857,5023.417824,3653.437583
2,2013,68509.832156,5281.641980,3819.969007
3,2014,66557.437750,5401.426941,3505.053449


In [36]:
query_dept_total = """
SELECT JobTitle, SUM(TotalPay) as TotalPay
FROM Salaries
WHERE JobTitle LIKE '%FIRE%' OR JobTitle LIKE '%POLICE%'
GROUP BY JobTitle
ORDER BY TotalPay DESC
LIMIT 10;
"""
dept_total = pd.read_sql(query_dept_total, conn)
dept_total

,JobTitle,TotalPay
0,Firefighter,3.441351e+08
1,Police Officer 3,3.266096e+08
2,Police Officer 2,1.558189e+08
3,Police Officer,1.421117e+08
4,EMT/Paramedic/Firefighter,1.222653e+08
5,FIREFIGHTER,1.105979e+08
6,"Lieutenant, Fire Suppression",1.037536e+08
7,POLICE OFFICER III,9.958299e+07
8,POLICE OFFICER I,5.331422e+07
9,SERGEANT III (POLICE DEPARTMENT),4.464028e+07


In [42]:
import ace_tools as tools
tools.display_dataframe_to_user(...)


ModuleNotFoundError: No module named 'ace_tools'

In [44]:
# Show the results in a clean way
print("Most Common Job Titles")
display(most_common_jobs)

print("Yearly Total Payments")
display(yearly_payments)

print("Top Earners")
display(top_earners)

print("Average Pay Types by Year")
display(avg_pay_types)

print("Top Paying Departments")
display(dept_total)


Most Common Job Titles


,JobTitle,Count
0,Transit Operator,7036
1,Special Nurse,4389
2,Registered Nurse,3736
3,Public Svc Aide-Public Works,2518
4,Police Officer 3,2421
5,Custodian,2418
6,TRANSIT OPERATOR,2388
7,Firefighter,2359
8,Recreation Leader,1971
9,Patient Care Assistant,1945


Yearly Total Payments


,Year,TotalPayment
0,2011,2.594195e+09
1,2012,2.724848e+09
2,2013,2.918656e+09
3,2014,2.876911e+09


Top Earners


,EmployeeName,JobTitle,TotalPayBenefits
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28
2,David Shinn,Deputy Chief 3,510732.68
3,Amy P Hart,Asst Med Examiner,479652.21
4,William J Coaker Jr.,Chief Investment Officer,436224.36
5,Gregory P Suhr,Chief of Police,425815.28
6,Joanne M Hayes-White,"Chief, Fire Department",422353.40
7,Gregory P Suhr,Chief of Police,418019.22
8,Joanne M Hayes-White,"Chief, Fire Department",417435.10
9,Ellen G Moffatt,Asst Med Examiner,415767.94


Average Pay Types by Year


,Year,AvgBasePay,AvgOvertimePay,AvgOtherPay
0,2011,63595.956517,4531.065429,3617.081926
1,2012,65436.406857,5023.417824,3653.437583
2,2013,68509.832156,5281.641980,3819.969007
3,2014,66557.437750,5401.426941,3505.053449


Top Paying Departments


,JobTitle,TotalPay
0,Firefighter,3.441351e+08
1,Police Officer 3,3.266096e+08
2,Police Officer 2,1.558189e+08
3,Police Officer,1.421117e+08
4,EMT/Paramedic/Firefighter,1.222653e+08
5,FIREFIGHTER,1.105979e+08
6,"Lieutenant, Fire Suppression",1.037536e+08
7,POLICE OFFICER III,9.958299e+07
8,POLICE OFFICER I,5.331422e+07
9,SERGEANT III (POLICE DEPARTMENT),4.464028e+07
